In [52]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import EditedNearestNeighbours,RandomUnderSampler
import imblearn

## 1.数据获取、预处理

In [84]:
from utils import *
dataset = pd.read_csv(r"C:\Users\jiangqi\Desktop\code_detect\Detecting-Cyber-Attacks-Kaggle-Callenge-master\data\partial_labels2.csv")
x,segments = dataset.axes
array = dataset.values    #array   50*100    每条  ‘user0’,0,0,^,1,0

labels=[]
for ele in array:
    label=ele[1:]
    labels.extend(label)

arr_all=[]
for testUser in range(0,50):  
    name='User{}'.format((testUser+1))
    z = np.reshape(array[testUser,1:],-1)    ##  100*1    (5001-15000,100)*1
    Y_train = z[0:50]
    Y_train = Y_train.astype(int)
    
    path_read = r"C:\Users\jiangqi\Desktop\code_detect\masquerade-data\{}".format(name)
    instructions = pd.read_csv(path_read, sep=" ", header=None)   ##该用户的commands    15000*1
    arr = np.reshape(instructions.values,(150,100))               ##将15000*1 转置  150*100
    arr_all.extend(arr[50:150])
arr_all=np.array(arr_all)

##处理   ''.join()

x=[]
for li  in arr_all:
    s=' '.join(li)
    x.append(s)
# x=np.array(x)
# labels=np.array(labels)

In [33]:
x[0]

'java .java_wr expr expr dirname basename egrep egrep egrep egrep egrep java aacdec cat aiffplay sh aacdec cat aiffplay sh aacdec cat aiffplay sh netscape netscape netscape netscape netscape netscape netscape aacdec cat aiffplay sh netscape netscape netscape netscape netscape netscape netscape netscape netscape netscape netscape netscape netscape netscape netscape hostname id nawk getopt true grep date lp find mkdir expr generic cat file post awk cat post rm generic ln ln generic lp sh getpgrp LOCK true ls sed FIFO cat date generic generic date generic gethost download tcpostio tcpostio tcpostio tcpostio cat generic ls generic date generic rm'

In [6]:
len(labels)

5000

In [8]:
len(arr_all)

5000

In [9]:
x_train,x_test,y_train,y_test=train_test_split(x,labels,train_size=0.7)

In [35]:
pd.Series(y_test).value_counts()

0    1419
1      81
dtype: int64

## 2.词向量化 

In [14]:
vectorizer = TfidfVectorizer(min_df=0.1,ngram_range=(2,4))  
vectorizer.fit_transform(x_train)

<3500x216 sparse matrix of type '<class 'numpy.float64'>'
	with 167243 stored elements in Compressed Sparse Row format>

In [15]:
x_train=vectorizer.transform(x_train)
x_test=vectorizer.transform(x_test)

## 3.分类预测

### 3.1XGBClassifier分类结果 作为baseline

In [63]:
clf=xgb.XGBClassifier(random_state=42)
clf.fit(x_train,y_train)
pre=clf.predict(x_test)
print('f1_score=',metrics.f1_score(y_test,pre,pos_label=0))
print('recall=',metrics.recall_score(y_test,pre,pos_label=0))
print('precision=',metrics.precision_score(y_test,pre,pos_label=0))
print('accuracy=',metrics.accuracy_score(y_test,pre))
print('混淆矩阵\n',metrics.confusion_matrix(y_test,pre))
print('G-mean=',imblearn.metrics.geometric_mean_score(y_test,pre,pos_label=0))

f1_score= 0.9722507708119219
recall= 1.0
precision= 0.946
accuracy= 0.946
混淆矩阵
 [[1419    0]
 [  81    0]]
G-mean= 0.0


### 3.2 调整样本权重  正样本：负样本=1：5

In [64]:
clf=xgb.XGBClassifier(scale_pos_weight=0.2,pos_label=0,random_state=42)
clf.fit(x_train,y_train)
pre=clf.predict(x_test)
print('f1_score=',metrics.f1_score(y_test,pre,pos_label=0))
print('recall=',metrics.recall_score(y_test,pre,pos_label=0))
print('precision=',metrics.precision_score(y_test,pre,pos_label=0))
print('accuracy=',metrics.accuracy_score(y_test,pre))
print('混淆矩阵\n',metrics.confusion_matrix(y_test,pre))
print('G-mean=',imblearn.metrics.geometric_mean_score(y_test,pre,pos_label=0))

f1_score= 0.9722507708119219
recall= 1.0
precision= 0.946
accuracy= 0.946
混淆矩阵
 [[1419    0]
 [  81    0]]
G-mean= 0.0


### 3.3 欠采样处理   
负样本/正样本=0.3（原比例为0.043） 

In [65]:
enn = RandomUnderSampler(0.3,random_state=0)    ##少数比多数 0.2
X_resampled, y_resampled = enn.fit_sample(x_train, y_train)

生成的新样本：正样本500例，负样本150例

In [66]:
pd.Series(y_resampled).value_counts()    

0    500
1    150
dtype: int64

In [67]:
clf=xgb.XGBClassifier()
clf.fit(X_resampled,y_resampled)
pre=clf.predict(x_test)
print('f1_score=',metrics.f1_score(y_test,pre,pos_label=0))
print('recall=',metrics.recall_score(y_test,pre,pos_label=0))
print('precision=',metrics.precision_score(y_test,pre,pos_label=0))
print('accuracy=',metrics.accuracy_score(y_test,pre))
print('混淆矩阵\n',metrics.confusion_matrix(y_test,pre))
print('G-mean=',imblearn.metrics.geometric_mean_score(y_test,pre,pos_label=0))

f1_score= 0.9570680628272252
recall= 0.9661733615221987
precision= 0.9481327800829875
accuracy= 0.918
混淆矩阵
 [[1371   48]
 [  75    6]]
G-mean= 0.2675227039893857


### 3.4过采样处理
处理后的负样本/正样本=0.6

In [79]:
sm=SMOTE(sampling_strategy=0.6,random_state=0)
X_resampled_smote, y_resampled_smote = sm.fit_sample(x_train, y_train)

In [80]:
pd.Series(y_resampled_smote).value_counts()

0    3350
1    2010
dtype: int64

In [81]:
clf=xgb.XGBClassifier()
clf.fit(X_resampled_smote,y_resampled_smote)
pre=clf.predict(x_test)
print(metrics.f1_score(y_test,pre,pos_label=0))
print(metrics.recall_score(y_test,pre,pos_label=0))
print(metrics.precision_score(y_test,pre,pos_label=0))
print(metrics.accuracy_score(y_test,pre))
print(metrics.confusion_matrix(y_test,pre))
print(imblearn.metrics.geometric_mean_score(y_test,pre,pos_label=0))

0.956552207428171
0.9619450317124736
0.9512195121951219
0.9173333333333333
[[1365   54]
 [  70   11]]
0.36143396420296714


## 4.进一步预处理

- 过采样，欠采样比较粗糙。合理怀疑词向量化

- linux 指令现实意义，ls  cd 可以作为停用词
- 前后有联系，词向量  n-gram 增大
- 使用RNN，LSTM 

In [93]:
comds=np.reshape(np.array(arr_all),-1)
pd.Series(comds).value_counts()[0:20]

sh          43418
netscape    21677
generic     21659
cat         21259
ls          20059
popper      17566
date        15002
sendmail    13518
rm          11893
sed         11139
nawk        10526
grep         9739
expr         9511
tcsh         8666
tcpostio     7334
true         6610
ln           6534
uname        6423
hostname     6158
gcc          5883
dtype: int64

In [ ]:
2_grame cangyong 

In [99]:
x_train[0]

<1x216 sparse matrix of type '<class 'numpy.float64'>'
	with 66 stored elements in Compressed Sparse Row format>

In [95]:
from nltk.probability import FreqDist

词集模型   
- 词set（） 后转dict行为


In [56]:
fdist = FreqDist(x).keys()   ##set后

NameError: name 'FreqDist' is not defined

In [116]:
x=np.array(x)

In [119]:
print(len(set(x)))
print(len(fdist))

4708

In [113]:
def get_user_cmd_feature_new(user_cmd_list,dist):
    user_cmd_feature=[]
    for cmd_list in user_cmd_list:
        s=[]
        for cmd in  cmd_list:
            v = [0] * len(dist)
            for i in range(0, len(dist)):
                if cmd == dist[i]:
                    v[i] = 1
            s.append(v)
        user_cmd_feature.append(s)
    return user_cmd_feature

In [124]:
user_cmd_feature=get_user_cmd_feature_new(x,fdist)

TypeError: 'dict_keys' object is not subscriptable